# Verify GPU

In [1]:
!nvidia-smi

Sat Apr 24 18:07:21 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   55C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Weight and Bias (Assisting Metrics, Optional)

In [2]:
!pip install wandb
!wandb login
project_name = "CoLA with BERT"  # @param {type:"string"}
import os

os.environ["WANDB_PROJECT"] = project_name

     |████████████████████████████████| 2.1MB 8.1MB/s 
     |████████████████████████████████| 163kB 30.0MB/s 
     |████████████████████████████████| 133kB 35.3MB/s 
     |████████████████████████████████| 102kB 12.8MB/s 
     |████████████████████████████████| 71kB 10.6MB/s 
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp37-none-any.whl size=6489 sha256=3dc43430b157ee8c54fd8b28cdd59253c516d561f7cf15100353b9c1bf950d78
  Stored in directory: /root/.cache/pip/wheels/68/39/1a/5e402bdfdf004af1786c8b853fd92f8c4a04f22aad179654d1
  Created wheel for pathtools: filename=pathtools-0.1.2-cp37-none-any.whl size=8786 sha256=b68f734f0cda29373ba41820bec520dccff34ddb613f41e608c465cda60ce0ff
  Stored in directory: /root/.cache/pip/wheels/0b/04/79/c3b0c3a0266a3cb4376da31e5bfe8bba0c489246968a68e843
Successfully built subprocess32 pathtools
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter: 
wandb: A

# Installation

In [3]:
!pip install transformers==4.5.1 datasets==1.5.0

     |████████████████████████████████| 2.1MB 7.2MB/s 
     |████████████████████████████████| 194kB 34.1MB/s 
     |████████████████████████████████| 901kB 33.9MB/s 
     |████████████████████████████████| 3.3MB 51.9MB/s 
     |████████████████████████████████| 245kB 51.2MB/s 
     |████████████████████████████████| 112kB 59.8MB/s 


In [4]:
import os

import numpy as np
from datasets import load_dataset, load_metric
from transformers import (
    BertConfig,
    BertForSequenceClassification,
    BertTokenizerFast,
    EvalPrediction,
    Trainer,
    TrainingArguments,
    default_data_collator,
)

In [5]:
per_device_train_batch_size = 32  # @param {type:"slider", min:1, max:64, step:1}
learning_rate = 2e-5  # @param {type:"slider", min:2e-7, max:2e-3, step:2e-7}
num_train_epochs = 3  # @param {type:"slider", min:1, max:10, step:1}
logging_steps = 10  # @param {type:"slider", min:10, max:100, step:5}

# Tokenizer

In [6]:
tokenizer = BertTokenizerFast.from_pretrained("bert-base-cased")

# Dataset

In [7]:
datasets = load_dataset("glue", "cola")
label_list = datasets["train"].features["label"].names

Dataset glue downloaded and prepared to /root/.cache/huggingface/datasets/glue/cola/1.0.0/7c99657241149a24692c402a5c3f34d4c9f1df5ac2e4c3759fadea38f6cb29c4. Subsequent calls will reuse this data.


In [8]:
def preprocess_function(examples):
    # Tokenize the texts
    result = tokenizer(
        examples["sentence"],
        padding="max_length",
        max_length=128,
        truncation="longest_first",
    )
    result["label"] = examples["label"]
    return result

In [9]:
datasets = datasets.map(preprocess_function, batched=True)
datasets["test"] = datasets["test"].remove_columns("label")

In [10]:
for index in range(3):
    print(f"Sample {index} of the training set: {datasets['train'][index]}.")

Sample 0 of the training set: {'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'idx': 0, 'input_ids': [101, 3458, 2053, 1281, 112, 189, 4417, 1142, 3622, 117, 1519, 2041, 1103, 1397, 1141, 1195, 17794, 119, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'label': 1, 'sentence': "Our friends won't buy this analysis, let alone the next one we propose.", 'toke

# Model

In [11]:
config = BertConfig.from_pretrained(
    "bert-base-cased", num_labels=len(label_list), finetuning_task="cola"
)
model = BertForSequenceClassification.from_pretrained("bert-base-cased", config=config)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

# Metric

In [12]:
metric = load_metric("glue", "cola")

In [13]:
def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
    preds = np.argmax(preds, axis=1)
    result = metric.compute(predictions=preds, references=p.label_ids)
    if len(result) > 1:
        result["combined_score"] = np.mean(list(result.values())).item()
    return result

# Trainer

In [14]:
training_args = TrainingArguments(
    output_dir="Transformers Trainer",
    do_train=True,
    do_eval=True,
    evaluation_strategy="epoch",
    per_device_train_batch_size=per_device_train_batch_size,
    learning_rate=learning_rate,
    num_train_epochs=num_train_epochs,
    logging_strategy="steps",
    logging_steps=logging_steps,
    report_to="wandb" if os.getenv("WANDB_PROJECT") else "none",
)

In [15]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=datasets["train"],
    eval_dataset=datasets["validation"],
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    data_collator=default_data_collator,
)

# Training

In [16]:
train_result = trainer.train()
metrics = train_result.metrics
metrics["train_samples"] = len(datasets["train"])
trainer.save_model()
trainer.log_metrics("train", metrics)
trainer.save_metrics("train", metrics)
trainer.save_state()

wandb: Currently logged in as: tomy0000000 (use `wandb login --relogin` to force relogin)


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:760: UserWarning: Using non-full backward hooks on a Module that does not return a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_output. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not return a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:795: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "


Epoch,Training Loss,Validation Loss,Matthews Correlation,Runtime,Samples Per Second
1,0.407100,0.445434,0.526242,8.855100,117.786000
2,0.321800,0.434242,0.598865,9.142200,114.087000
3,0.169600,0.505683,0.590512,8.904800,117.128000


# Evaluation

In [17]:
metrics = trainer.evaluate()
metrics["eval_samples"] = len(datasets["validation"])
trainer.log_metrics("eval", metrics)
trainer.save_metrics("eval", metrics)

# Test

## On dataset

In [18]:
predictions = trainer.predict(test_dataset=datasets["test"]).predictions
predictions = np.argmax(predictions, axis=1)

In [19]:
for index, (sample, pred) in enumerate(zip(datasets["test"]["sentence"], predictions)):
    print(f"{index}\t{label_list[pred]}\t{sample}")

0	acceptable	Bill whistled past the house.
1	acceptable	The car honked its way down the road.
2	acceptable	Bill pushed Harry off the sofa.
3	acceptable	the kittens yawned awake and played.
4	acceptable	I demand that the more John eats, the more he pay.
5	acceptable	If John eats more, keep your mouth shut tighter, OK?
6	acceptable	His expectations are always lower than mine are.
7	acceptable	The sooner you call, the more carefully I will word the letter.
8	acceptable	The more timid he feels, the more people he interviews without asking questions of.
9	acceptable	Once Janet left, Fred became a lot crazier.
10	acceptable	If you give every senator enough opportunity, he will succumb to corruption.
11	acceptable	The more time that any senator spends with lobbyists, the more likely he is to succumb to corruption.
12	unacceptable	Which problem does the sooner you solve, the more easily you'll satisfy the folks up at corporate headquarters?
13	acceptable	Which problem do you think that the soo

## Manually

In [20]:
sentence = "The probable hostile German reaction is unfortunate."  # @param {type:"string"}
tokenized_input = tokenizer(sentence, return_tensors="pt").to(model.device)
outputs = model(**tokenized_input)
print(f"Prediction: {label_list[outputs.logits.argmax(dim=-1).item()]}")

Prediction: acceptable


# Inference

In [1]:
import torch
from transformers import BertConfig, BertForSequenceClassification, BertTokenizerFast

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
label_list = ["unacceptable", "acceptable"]
tokenizer = BertTokenizerFast.from_pretrained("Transformers Trainer")
config = BertConfig.from_pretrained("Transformers Trainer", finetuning_task="cola")
model = BertForSequenceClassification.from_pretrained(
    "Transformers Trainer", config=config
).to(device)

In [3]:
sentence = "The probable hostile German reaction is unfortunate."  # @param {type:"string"}
tokenized_input = tokenizer(sentence, return_tensors="pt").to(device)
outputs = model(**tokenized_input)
print(f"Prediction: {label_list[outputs.logits.argmax(dim=-1).item()]}")

Prediction: acceptable
